## Callbacks - Learning Rate
- 처음에는 크게 했다가 점점 줄이는 방법으로 많이 사용함
- epoch 마다 조금씩 줄임

강사 팁
- 처음에는 0.01 (좀 큰 숫자)로 학습을 한 결과 곡선을 보고 학습이 잘 안되는 epoch를 확인하고 그 뒤로 부터 learning rate를 조정 함.

In [36]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt
%matplotlib inline

import math

## Hyperparameter Tunning

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Data Preprocess

In [14]:
train_paths = glob('../dataset/cifar/train/*.png')[:100]
test_paths = glob('../dataset/cifar/test/*.png')[:100]

In [15]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [16]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [17]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [18]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [19]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [20]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [22]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Learning Rate Scheduler

In [37]:
# 매 epoch learning rate를 변경해 준다.
def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10 - epoch))

In [42]:
# verbose를 1로 해주면 learing rate를 줄일때 마다 로그로 보여준다.
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)
#learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

## Training
- http://localhost:6006

In [43]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[learning_rate_scheduler]
)

Epoch 1/20
3/3 [==============================] - 1s 234ms/step - loss: 0.8778 - accuracy: 0.6471 - val_loss: 2.8514 - val_accuracy: 0.3333
Epoch 2/20
3/3 [==============================] - 1s 260ms/step - loss: 1.2165 - accuracy: 0.5729 - val_loss: 2.8243 - val_accuracy: 0.2292
Epoch 3/20
3/3 [==============================] - 1s 188ms/step - loss: 0.6107 - accuracy: 0.6500 - val_loss: 2.8834 - val_accuracy: 0.2292
Epoch 4/20
3/3 [==============================] - 1s 250ms/step - loss: 1.2546 - accuracy: 0.5729 - val_loss: 2.8488 - val_accuracy: 0.2812
Epoch 5/20
3/3 [==============================] - 1s 232ms/step - loss: 0.6826 - accuracy: 0.6765 - val_loss: 3.0460 - val_accuracy: 0.2812
Epoch 6/20
3/3 [==============================] - 1s 233ms/step - loss: 0.8517 - accuracy: 0.5441 - val_loss: 3.0468 - val_accuracy: 0.3021
Epoch 7/20
3/3 [==============================] - 1s 248ms/step - loss: 1.1516 - accuracy: 0.6042 - val_loss: 2.9316 - val_accuracy: 0.2708
Epoch 8/20
3/3 [====